## DQN-Based Heating and Ventilation System.

### Problem Assumptions.

First asumption: 

The server temperature is apporximated since I don't have hestorical data of the server temperature. I apporximate the temperature through the multiple linear regression equation:

<em>Server temperature</em> =  <em>$ b_{\mathrm{0}\;}$ +  $ b_{\mathrm{1}\;}$ x Atmospheric temperature +  $ b_{\mathrm{2}\;}$ x number of users +  $ b_{\mathrm{3}\;}$ x Rate of data transmission.</em>


 <em>Where:</em> $ b_{\mathrm{0}\;}$ ${\in}$  ${\rm I\!R}$, $ b_{\mathrm{1}\;} > 0,\hspace{.1cm}  b_{\mathrm{2}\;} > 0,\hspace{.1cm}  b_{\mathrm{3}\;} > 0$. 


Assuming we performe the multiple linear regression and I get the values : 

$$
b_{\mathrm{0}\;} = 0,\hspace{.2cm}
b_{\mathrm{1}\;} = 1, \hspace{.2cm}
b_{\mathrm{2}\;} = 1.25\hspace{.2cm}
b_{\mathrm{3}\;} = 1.25
$$

then the equation becomes: 

<em>Server temperature</em> =  <em> Atmospheric temperature +  $ 1.25$ x number of users +  $ 1.25$ x Rate of data transmission.</em>

Second assumption: 

The cost of energy needed to bring back the server into the optimal temperature are approximated also with a linear regression. 

$$
e_{\mathrm{hv}\;} = \alpha \vert T_{\mathrm{t+1}\;}-T_{\mathrm{t}\;}  \vert + \beta
$$

$\newline$ 

<em> where: 



$e_{\mathrm{hv}\;}$ : is the energy spent either for heating or ventilation to bring back the server to optimal range of temperature. 

$\vert T_{\mathrm{t+1}\;}-T_{\mathrm{t}\;}  \vert$ : is the temperature change in the server between t and t+1.

$\newline$ 

$\alpha > 0, \hspace{.2cm} \beta \in {\rm I\!R}$

and indeed we dont't have reel temperature data so I assume that the value after performing the linear regression :

$\alpha = 1 \hspace{.2cm}and\hspace{.2cm}  \beta = 0$.

therefore the assumption becomes : $e_{\mathrm{hv}\;} = \alpha \vert T_{\mathrm{t+1}\;}-T_{\mathrm{t}\;}  \vert$.



### Importing libraries.


In [1]:
# IMPORTING LIBRARIES AND PACKAGES.
import os
import numpy as np
import random as rn
from src.models.environment import Environment
from src.models.nn_dropout import Network
from src.models.dqn import DQN
from keras.models import load_model

### Training the AI with Early Stopping.

In [2]:
# SETTING SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)

# SETTING THE PARAMETERS
epsilon = .3
number_actions = 5
direction_boundary = (number_actions - 1) / 2
number_epochs = 50
max_memory = 3000
batch_size = 512
temperature_step = 1.5

# BUILDING THE ENVIRONMENT BY SIMPLY CREATING AN OBJECT OF THE ENVIRONMENT CLASS
env = Environment(optimal_temperature = (18.0, 24.0), initial_month = 0, initial_number_users = 20, initial_rate_data = 30)

# BUILDING THE NEURAL NETWORK BY SIMPLY CREATING AN OBJECT OF THE NETWORK CLASS
Network = Network(learning_rate = 0.5, number_actions = number_actions)

# BUILDING THE DQN MODEL BY SIMPLY CREATING AN OBJECT OF THE DQN CLASS
dqn = DQN(max_memory = max_memory, discount = 0.9)

# CHOOSING THE MODE
train = True

# TRAINING THE AI
env.train = train
model = Network.model
early_stopping = True
patience = 10
best_total_reward = -np.inf
patience_count = 0
if (env.train):
    # STARTING THE LOOP OVER ALL THE EPOCHS (1 Epoch = 5 Months)
    for epoch in range(1, number_epochs):
        # INITIALIAZING ALL THE VARIABLES OF BOTH THE ENVIRONMENT AND THE TRAINING LOOP
        total_reward = 0
        loss = 0.
        new_month = np.random.randint(0, 12)
        env.reset(new_month = new_month)
        out_range = False
        current_state, _, _ = env.observe()
        timestep = 0
        # STARTING THE LOOP OVER ALL THE TIMESTEPS (1 Timestep = 1 Minute) IN ONE EPOCH
        while ((not out_range) and timestep <= 5 * 30 * 24 * 60):
            # PLAYING THE NEXT ACTION BY EXPLORATION
            if np.random.rand() <= epsilon:
                action = np.random.randint(0, number_actions)
                if (action - direction_boundary < 0):
                    direction = -1
                else:
                    direction = 1
                energy_ai = abs(action - direction_boundary) * temperature_step
            # PLAYING THE NEXT ACTION BY INFERENCE
            else:
                q_values = model.predict(current_state)
                action = np.argmax(q_values[0])
                if (action - direction_boundary < 0):
                    direction = -1
                else:
                    direction = 1
                energy_ai = abs(action - direction_boundary) * temperature_step
            # UPDATING THE ENVIRONMENT AND REACHING THE NEXT STATE
            next_state, reward, out_range = env.update_env(direction, energy_ai, ( new_month + int(timestep/(30*24*60)) ) % 12)
            total_reward += reward
            # STORING THIS NEW TRANSITION INTO THE MEMORY
            dqn.remember([current_state, action, reward, next_state], out_range)
            # GATHERING IN TWO SEPARATE BATCHES THE INPUTS AND THE TARGETS
            inputs, targets = dqn.get_batch(model, batch_size = batch_size)
            # COMPUTING THE LOSS OVER THE TWO WHOLE BATCHES OF INPUTS AND TARGETS
            loss += model.train_on_batch(inputs, targets)
            timestep += 1
            current_state = next_state
        # PRINTING THE TRAINING RESULTS FOR EACH EPOCH
        print("\n")
        print("Epoch: {:03d}/{:03d}".format(epoch, number_epochs))
        print("Total Energy spent with an AI: {:.0f}".format(env.total_energy_ai))
        print("Total Energy spent with no AI: {:.0f}".format(env.total_energy_noai))
        # EARLY STOPPING
        if (early_stopping):
            if (total_reward <= best_total_reward):
                patience_count += 1
            elif (total_reward > best_total_reward):
                best_total_reward = total_reward
                patience_count = 0
            if (patience_count >= patience):
                print("Early Stopping")
                break
        # SAVING THE MODEL
        model.save(".\src\models\hvs.h5")

c:\Users\35841\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 0s 32ms/step


Epoch: 001/050
Total Energy spent with an AI: 2
Total Energy spent with no AI: 0
1/1 [==============================] - 0s 35ms/step


Epoch: 002/050
Total Energy spent with an AI: 4
Total Energy spent with no AI: 26
1/1 [==============================] - 0s 38ms/step


Epoch: 003/050
Total Energy spent with an AI: 27
Total Energy spent with no AI: 118
1/1 [==============================] - 0s 48ms/step


Epoch: 004/050
Total Energy spent with an AI: 0
Total Energy spent with no AI: 0
1/1 [==============================] - 0s 65ms/step


Epoch: 005/050
Total Energy spent with an AI: 4
Total Energy spent with no AI: 22
1/1 [==============================] - 0s 217ms/step


Epoch: 006/050
Total Energy spent with an AI: 28
Total Energy spent with no AI: 116
1/1 [==============================] - 0s 60ms/step


### Testing the model.

In [ ]:
# SETTING THE PARAMETERS
number_actions = 5
direction_boundary = (number_actions - 1) / 2
temperature_step = 1.5

# BUILDING THE ENVIRONMENT BY SIMPLY CREATING AN OBJECT OF THE ENVIRONMENT CLASS
env = Environment(optimal_temperature = (18.0, 24.0), initial_month = 0, initial_number_users = 20, initial_rate_data = 30)

# LOADING A PRE-TRAINED NEURAL NETWORK
model = load_model(".\src\models\hvs.h5")

# CHOOSING THE MODE
train = False

# RUNNING A 1 YEAR SIMULATION IN INFERENCE MODE
env.train = train
current_state, _, _ = env.observe()
for timestep in range(0, 12 * 30 * 24 * 60):
    q_values = model.predict(current_state)
    action = np.argmax(q_values[0])
    if (action - direction_boundary < 0):
        direction = -1
    else:
        direction = 1
    energy_ai = abs(action - direction_boundary) * temperature_step
    next_state, reward, out_range = env.update_env(direction, energy_ai, int(timestep / (30 * 24 * 60)))
    current_state = next_state

# PRINTING THE TRAINING RESULTS FOR EACH EPOCH
print("\n")
print("Total Energy spent with an AI: {:.0f}".format(env.total_energy_ai))
print("Total Energy spent with no AI: {:.0f}".format(env.total_energy_noai))
print("ENERGY SAVED: {:.0f} %".format((env.total_energy_noai - env.total_energy_ai) / env.total_energy_noai * 100))